In [1]:
import os
from utils import *
import optuna

In [7]:
data, usermap, itemmap, users = load_data2()
data_train, data_test, data_val = split_data2(data, 0, 0.2)

In [ ]:
study_name = "ImplicitALS"
study = optuna.create_study(
    study_name=study_name,
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

In [ ]:
from Evaluation.Evaluator import EvaluatorHoldout
from implicit.gpu.als import AlternatingLeastSquares as IALSRecommender
from implicit.evaluation import ranking_metrics_at_k

evaluator_validation = EvaluatorHoldout(data_val, cutoff_list=[10])

def objective(trial):
    num_factors = trial.suggest_int('num_factors', 1, 500)
    reg = trial.suggest_float('reg', 1e-5, 1, log=True)
    alpha = trial.suggest_float('alpha', 1e-7, 100, log=True)
    iterations = trial.suggest_categorical('iterations', [10, 20, 50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000])
    
    recommender = IALSRecommender(
        factors=num_factors,
        regularization=reg,
        alpha=alpha,
        iterations=iterations,
        calculate_training_loss=False,
    )
    recommender.fit(
        data_train,
        show_progress=False,
    )
    
    return ranking_metrics_at_k(recommender, data_train, data_val, K=10, show_progress=False)['map']

study.optimize(objective, n_trials=300)

In [ ]:
study.best_params

In [ ]:
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender

recommender = IALSRecommender(data_train, verbose=False)
recommender.fit(**study.best_params)

In [ ]:
pr, re, ev_map, ev_ndcg, ev_auc = evaluator(recommender, data_train, data_val)
print("Precision: {:.8f}".format(pr))
print("Recall: {:.8f}".format(re))
print("MAP: {:.8f}".format(ev_map))

In [ ]:
submission2(recommender, users, usermap, itemmap, data_train)